In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
OPENAI_API_KEY = ""
ANTHROPIC_API_KEY = ""

In [ ]:
import os

# Set your API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# Initialize OpenAI Chat Model
llm_openai = ChatOpenAI(model="gpt-4o")
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")


## 1. Constructing the basic langGraph flow

In [ ]:
from typing import Annotated, List

from typing_extensions import TypedDict
from langchain.schema import HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[List[AIMessage | HumanMessage], add_messages]

graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State) -> State:
    bot_response = llm.invoke(state["messages"])
    print(state["messages"]+[bot_response])
    print("\n")
    return {"messages": [bot_response]}


# The first argument is the unique node name
# The second argument is the function or object that will be called
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

### Show the visual graph node

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Run the chatbot using "graph.stream"

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

# value["messages"][-1].content
# it is used to access the content of the last message 
# in a list stored under the key "messages" in a dictionary named value

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## 2. Let's enhance it where we will keep the conversation state using "graph.invoke"

In [ ]:
from typing import TypedDict, List, Annotated
from langchain.schema import HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[List[AIMessage | HumanMessage], add_messages]
    user_input: str

In [ ]:
state = AgentState(messages=[], user_input="")
print(f"Initial state: {state}")

### Now let's add functions to modify the state

In [ ]:
import openai

# 1. Function to take the user input and store it in AgentState
def add_user_message(state: AgentState) -> AgentState:
    new_query = HumanMessage(content=user_input)
    return {"messages":[new_query], # Append new message
           "user_input": ""}  # set back user input into None


# 2. Function to call LLM and get response
def generate_ai_response(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])  # Call LLM API
    return {"messages":[response], "user_input": ""}  # Append AI response


### Let's create the graph instance to put together all the states and the functionalities

In [ ]:
# 1. Initiate the graph instance from the main class StateGraph that hooks up the data stored in AgentState class
graph = StateGraph(AgentState)

# 2. Define nodes
graph.add_node("add_user_message", add_user_message)
graph.add_node("generate_ai_response", generate_ai_response)


# 3. Define edges (flow of the graph)
graph.add_edge(START, "add_user_message")
graph.add_edge("add_user_message", "generate_ai_response")
graph.add_edge("generate_ai_response", END)

# 4. Convert the graph structure into an executable flow
workflow = graph.compile() 

### Let's test and run the workflow

In [ ]:
# Initialize chatbot state and must the same class AgenState!
state = AgentState(messages=[], user_input="")
print(f"\nInitial state: {state}")


# Simulate conversation
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
        
    state["user_input"] = user_input
    state = workflow.invoke(state)  # Run the workflow and update state
    bot_response = state["messages"][-1].content  # Get last AI response
    print(f"Bot: {bot_response}")
    
    print(f"\nUpdated state: {state}") # Checkpoint to check the updated state